In [97]:
from collections import Counter
import numpy as np
import trino
import pandas as pd
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
##mysql connector
import mysql.connector
myconn = mysql.connector.connect(host = "tango-read-nifi.ci9kqv9mlekc.ap-south-1.rds.amazonaws.com",
                                 user = "pikachu",passwd = "crofarm123abc",database = "consumer")
cur_mysql = myconn.cursor()


##trino connector
from trino.dbapi import connect
conn = connect(host="presto.crofarm.com",port=443,catalog='hive',http_scheme='https',auth=trino.auth.BasicAuthentication("db.manager","crofarm"))
cur_trino = conn.cursor()


trino_final_tables=[]
schema=["consumer","crofarmusers","products"]
for i in range(len(schema)):
    b="show tables from "+schema[i]
    cur_trino.execute(b)
    trino_tables=cur_trino.fetchall()
    for j in range(len(trino_tables)):
        trino_final_tables.append(str(schema[i].lower())+"."+str(trino_tables[j][0].lower()))

trino_final_tables.sort()

mysql_final_tables=[]
schema=["consumer","crofarmUsers","products"]
for i in range(len(schema)):
    b="show tables from "+schema[i]
    cur_mysql.execute(b)
    mysql_tables=cur_mysql.fetchall()
    for j in range(len(mysql_tables)):
        mysql_final_tables.append(str(schema[i].lower())+"."+str(mysql_tables[j][0].lower()))

mysql_final_tables.sort()

    
not_present_in_trino=list(set(mysql_final_tables)-set(trino_final_tables))
not_present_in_trino.sort()
df=pd.DataFrame(not_present_in_trino,columns=['Table_not_present_in_trino'])
df['Full_load']=''
df['Incremental']=''
df['Update']=''
df['if_update_minimum_days_for_update']=''

In [98]:
df

,Table_not_present_in_trino,Full_load,Incremental,Update,if_update_minimum_days_for_update
0,consumer.agent_reactivation_share,,,,
1,consumer.auth_group,,,,
2,consumer.auth_group_permissions,,,,
3,consumer.auth_user,,,,
4,consumer.auth_user_groups,,,,
5,consumer.auth_user_user_permissions,,,,
6,consumer.cn_agent_delivery_rating_performance,,,,
7,consumer.cn_consumer_servicable_history,,,,
8,consumer.cn_consumer_subscribed_history,,,,
9,consumer.cn_invite,,,,


In [91]:
present=[]
not_present=[]
for i in range(len(not_present_in_trino)):
    try:
        cur_trino.execute("select count(*) from "+not_present_in_trino[i])
        present.append(cur_trino.fetchall())
    except:
        not_present.append('hi')
if(len(not_present)==len(not_present_in_trino)):
    recipients = ['agrim.gupta@crofarm.com'] 
    emaillist = [elem.strip().split(',') for elem in recipients]
    msg = MIMEMultipart()
    msg['Subject'] = "Tables not present in nifi"
    msg['From'] = 'agrim.gupta@crofarm.com'

    html = """\
            <html>
              <head></head>
              <body>
                  Please fill the types of tables and send to the data team 
                {0}
              </body>
            </html>
    """.format(df.to_html(index=False))

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    try:
        """Checking for connection errors"""

        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.ehlo()#NOT NECESSARY
        server.starttls()
        server.ehlo()#NOT NECESSARY
        server.login('agrim.gupta@crofarm.com','ckotcpsrixsfughy')
        server.sendmail(msg['From'], emaillist , msg.as_string())
        server.close()

    except Exception as e:
        print("Error for connection: {}".format(e))
else:
    recipients = ['agrim.gupta@crofarm.com'] 
    emaillist = [elem.strip().split(',') for elem in recipients]
    msg = MIMEMultipart()
    msg['Subject'] = "dataframe table TEST FROM GMAIL"
    msg['From'] = 'agrim.gupta@crofarm.com'

    html = """\
            <html>
              <head></head>
              <body>
                {some thing went wrong please check}
              </body>
            </html>
    """

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    try:
        """Checking for connection errors"""

        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.ehlo()#NOT NECESSARY
        server.starttls()
        server.ehlo()#NOT NECESSARY
        server.login('agrim.gupta@crofarm.com','ckotcpsrixsfughy')
        server.sendmail(msg['From'], emaillist , msg.as_string())
        server.close()

    except Exception as e:
        print("Error for connection: {}".format(e))

,Table_not_present_in_trino,Full_load,incremental,update
0,consumer.agent_reactivation_share,,,
1,consumer.auth_group,,,
2,consumer.auth_group_permissions,,,
3,consumer.auth_user,,,
4,consumer.auth_user_groups,,,
5,consumer.auth_user_user_permissions,,,
6,consumer.cn_agent_delivery_rating_performance,,,
7,consumer.cn_consumer_servicable_history,,,
8,consumer.cn_consumer_subscribed_history,,,
9,consumer.cn_invite,,,
